#Music Recommendation System

Ngo Minh Tan - For CS50X

16/10/23

Moscow - Russia




**INTRODUCTION**

Music recommendation systems are used to suggest songs or playlists to users based on their listening history or preferences. They are a popular type of recommendation system, and are used by many music streaming services, such as Spotify and Apple Music.

A typical music recommendation system using machine learning in the real world works in the following steps:

1. Data collection: The system collects data on users's listening habits, such as the songs they play, the playlists they create, and the artists they follow.
2. Feature extraction: The system extracts features from the music data, such as the genre, tempo, rhythm, and mood of the songs.
3. Model building: The system uses a machine learning algorithm to build a model that can predict which songs a user is likely to like, based on their listening history and the features of the music.
4. Recommendation generation: The system uses the model to generate a list of recommended songs for the user.

As for the sake of the final project/educational purpose, the implementation will be simplied from how the system actual work.





**IMPLEMENTATION**

Based on the project, there are serveral questions we need to answer


* What's the goal of our system and where we'll take the data? (recommend base on real playlists provided from APIs like Spotipy, recommend base on a datasets, recommend base on the sound provided...)
* How we'll approach this problem (using the lyrics, using the metadata/characteristics of the songs,...)
* What algorithms we'll use(TF-IDF, Collaborative filtering, Content-base filtering, Hybrid filtering, Clustering...)

After considering, I decided that I'll take data from a dataset, instead of the playlist-id approach because it's more fit for the project. For the algorithm, I will use cosine similarity


#EXPLORE DATASET

The dataset I chose is: https://www.kaggle.com/datasets/jashanjeetsinghhh/songs-dataset

In [15]:
#Import library
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime


import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '28339b556a91431ea6ea47939c150dce'
CLIENT_SECRET = '083b618b22be444f9348068f60d190a9'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
df = pd.read_csv('dataset.csv')
df.head(10)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,...,-8.807,1,0.1050,0.2890,0.000000,0.1890,0.6660,98.017,4,acoustic
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,...,-8.822,1,0.0355,0.8570,0.000003,0.0913,0.0765,141.284,3,acoustic
7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,...,-9.331,1,0.0417,0.5590,0.000000,0.0973,0.7120,150.960,4,acoustic
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,...,-8.700,1,0.0369,0.2940,0.000000,0.1510,0.6690,130.088,4,acoustic
9,9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.6320,...,-6.770,1,0.0295,0.4260,0.004190,0.0735,0.1960,78.899,4,acoustic


In [ ]:
df.shape

(114000, 21)

In [ ]:
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [ ]:
#Check if have any rows that are null
print(df.isnull().sum())

Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [ ]:
# drop the rows that are null
df = df.dropna()
print(df.isnull().sum())
df.shape

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


(113999, 21)

In [ ]:
df[df.duplicated('track_id')==True]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre


In [ ]:
df[df['track_id']=='71dLJx3qHOTQMTvvoE2dmd']


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
80966,4932,71dLJx3qHOTQMTvvoE2dmd,Bethel Music;Amanda Cook,Peace,King of My Heart,48,330840,False,0.515,0.0736,...,-15.492,1,0.031,0.975,0.000261,0.082,0.0684,109.964,4,ambient


In [ ]:
dfDistinct = df.drop_duplicates('track_id')
dfDistinct = dfDistinct.sort_values('track_id')
dfDistinct = dfDistinct.reset_index(drop=True)
df = dfDistinct


In [ ]:
df.shape

(89740, 21)

In [ ]:
df.to_csv('df.csv', index=False)



In [ ]:
df = pd.read_csv('df.csv')


In [ ]:
print(df['track_id'])

0        0000vdREvCVMxbQTkS888c
1        000CC8EParg64OmTxVnZ0p
2        000Iz0K615UepwSJ5z2RE5
3        000RDCYioLteXcutOjeweY
4        000qpdoc97IMTBvF8gwcpy
                  ...          
89735    7zxHiMmVLt4LGWpOMqOpUh
89736    7zxpdh3EqMq2JCkOI0EqcG
89737    7zyYmIdjqqiX6kLryb7QBx
89738    7zybSU9tFO9HNlwmGF7stc
89739    7zz7iNGIWhmfFE7zlXkMma
Name: track_id, Length: 89740, dtype: object


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Create a Spotipy object
sp = spotipy.Spotify(auth=access_token)

# Get the track information
track = sp.track('000CC8EParg64OmTxVnZ0p')

# Get the release date
release_date = track['album']['release_date']

# Print the release date
print(release_date)


2021-02-11


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
def get_release_date(dataset, access_token ):
    sp = spotipy.Spotify(auth=access_token)
    release_dates = []
    for trackid in dataset:
        info = sp.track(trackid)
        time.sleep(2)
        release_date = info['album']['release_date']
        if release_date is not None:
          release_dates.append(release_date)
    return release_dates

In [ ]:
df10= df['track_id'].head(10)
print(df10)

0    0000vdREvCVMxbQTkS888c
1    000CC8EParg64OmTxVnZ0p
2    000Iz0K615UepwSJ5z2RE5
3    000RDCYioLteXcutOjeweY
4    000qpdoc97IMTBvF8gwcpy
5    0017XiMkqbTfF2AUOzlhj6
6    001APMDOl3qtx1526T11n1
7    001YQlnDSduXd5LgBd66gT
8    001pyq8FLNSL1C8orNLI0b
9    002qpSULhHAw6DGqFxbaO1
Name: track_id, dtype: object


In [ ]:
release_dates = get_release_date(df10, access_token)
print(release_dates)

In [ ]:
# Identify the rows that have the value 'YOASOBI' in the 'artist' column
rows_to_print = df[df['artists'].isin(['YOASOBI'])].index

# Select the identified rows
df_filtered = df.loc[rows_to_print]

# Print the selected rows
print(df_filtered['track_name'])

1197                 怪物
1555              ハルジオン
3681     Into The Night
5315                 群青
7267            Romance
13025             ラブレター
13347             優しい彗星
22928                群青
25759               三原色
29409              ミスター
32593            もう少しだけ
33747                怪物
33789    Into The Night
36262               たぶん
37465               三原色
41919             夜に駆ける
47124             アンコール
48071          あの夢をなぞって
51047         もしも命が描けたら
59858               ハルカ
61635             優しい彗星
63500         もしも命が描けたら
67146          あの夢をなぞって
69450               たぶん
73181             夜に駆ける
79968               ハルカ
81595               好きだ
83966             ハルジオン
84776           Monster
85079                祝福
Name: track_name, dtype: object


#Demo the algorithms


In [2]:
#Load the database that has year column
df = pd.read_csv('final.csv')
df.head(10)

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,release
0,0000vdREvCVMxbQTkS888c,Rill,Lolly,Lolly,44,160725,True,0.910,0.374,8,...,0,0.1990,0.075700,0.003010,0.1540,0.432,104.042,4,german,2022-09-23
1,000CC8EParg64OmTxVnZ0p,Glee Cast,Glee Love Songs,It's All Coming Back To Me Now (Glee Cast Vers...,47,322933,False,0.269,0.516,0,...,1,0.0366,0.406000,0.000000,0.1170,0.341,178.174,4,club,2021-02-11
2,000Iz0K615UepwSJ5z2RE5,Paul Kalkbrenner;Pig&Dan,X,Böxig Leise - Pig & Dan Remix,22,515360,False,0.686,0.560,5,...,0,0.0462,0.001140,0.181000,0.1110,0.108,119.997,4,minimal-techno,2014-05-30
3,000RDCYioLteXcutOjeweY,Jordan Sandhu,Teeje Week,Teeje Week,62,190203,False,0.679,0.770,0,...,1,0.1900,0.058300,0.000000,0.0825,0.839,161.721,4,hip-hop,2018-02-12
4,000qpdoc97IMTBvF8gwcpy,Paul Kalkbrenner,Zeit,Tief,19,331240,False,0.519,0.431,6,...,0,0.0291,0.000964,0.720000,0.0916,0.234,129.971,4,minimal-techno,2001-10-08
5,0017XiMkqbTfF2AUOzlhj6,Chad Daniels,Busy Being Awesome,Thanksgiving Chicken,24,127040,True,0.536,0.780,5,...,0,0.9450,0.792000,0.000000,0.7350,0.452,173.912,3,comedy,2009-01-27
6,001APMDOl3qtx1526T11n1,Pink Sweat$;Kirby,New RnB,Better,0,176320,False,0.613,0.471,1,...,0,0.1070,0.316000,0.000001,0.1170,0.406,143.064,4,chill,2022-10-21
7,001YQlnDSduXd5LgBd66gT,Soda Stereo,Soda Stereo (Remastered),El Tiempo Es Dinero - Remasterizado 2007,38,177266,False,0.554,0.921,2,...,1,0.0758,0.019400,0.088100,0.3290,0.700,183.571,1,punk-rock,1984-02-06
8,001pyq8FLNSL1C8orNLI0b,Old Crow Medicine Show,O.C.M.S.,Poor Man,30,214600,False,0.580,0.290,2,...,1,0.0272,0.261000,0.000000,0.1250,0.497,91.321,4,bluegrass,2004-01-17
9,002qpSULhHAw6DGqFxbaO1,Tokyo Ghetto Pussy,Disco 2001,Love Generation,17,410666,False,0.531,0.950,9,...,0,0.0433,0.001220,0.826000,0.0613,0.553,159.974,4,happy,1995


In [3]:
#Recommend based on genre
def genre_recommendations(genre, num_recommendations=6):
    genre = genre[0].lower() + genre[1:]
    print(genre)
    same_genre = df.loc[df['track_genre'] == genre]
    if same_genre.empty:
        return
    same_genre = df.loc[df['track_genre'] == genre].sample(num_recommendations)
    return same_genre

print(genre_recommendations('J-rock'))

j-rock
                     track_id         artists  \
7760   0g3Nep5KmsTCCaw7GfWTKW          mol-74   
53155  4cZh6Ys8MeCrd9JKQpIxaA  go!go!vanillas   
76459  6dkNzag9ucTnvnhJ4QlI3f     Mr.Children   
38760  3Mk4nxxKHem8rWxdWLC0R6  Eikichi Yazawa   
58469  55Ww4Pa1iIQMhh0MLMetjo     Masaki Suda   
86845  7k7Bx4DdM56SiGWEVEK7iU         Onze:20   

                                              album_name  \
7760                                             Answers   
53155                                              FOOLs   
76459                                            I'LL BE   
38760  LIVE DECADE 1990-1999 (50th Anniversary Remast...   
58469                                               LOVE   
86845                                         Tudo Nosso   

                                            track_name  popularity  \
7760                                           Answers          40   
53155                                            サクラサク          32   
76459             

In [5]:
#Convert the date
def convert_release_date(release_date):

    try:
        # Check if the release_date is in %Y format.
        release_date_datetime = datetime.strptime(release_date, "%Y")
    except ValueError:
        # If it is not in %Y format, return the original release_date.
        return release_date

    # If the release_date is in %Y format, convert it to year_01_01 format.
    release_date_datetime = release_date_datetime.replace(month=1, day=1)
    release_date = release_date_datetime.strftime("%Y-%m-%d")

    return release_date


def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = convert_release_date(release_date)
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight


In [6]:
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = convert_release_date(release_date)
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [8]:
# Normalize the music features using Min-Max scaling
"""https://towardsdatascience.com/everything-you-need-to-know-about-min-max-normalization-in-python-b79592732b79"""
scaler = MinMaxScaler()
music_features = df[['danceability', 'energy', 'key', 'mode',
                     'loudness',  'speechiness', 'acousticness',
                     'instrumentalness', 'liveness', 'valence', 'tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [9]:
#Recommend based on features
def content_based_recommendations(input_song_name, input_artist_name, num_recommendations=5):

    # Get the indices of the songs in the dataset that match the input song name.

    song = (df['track_name'].str.contains(
        input_song_name, case=False, na=False, regex=False))
    art = (df['artists'].str.contains(
        input_artist_name, case=False, na=False, regex=False))

    matching_song_indices = df[song & art].index
    # If no songs match the input song name, return an empty list.
    if len(matching_song_indices) == 0:
        return

    # Get the index of the input song in the dataset.
    input_song_index = matching_song_indices[0]

    """ https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599"""

    # Calculate the similarity scores based on music features (cosine similarity).
    similarity_scores = cosine_similarity(
        [music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs.
    similar_song_indices = similarity_scores.argsort(
    )[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering.
    content_based_recommendations = df.iloc[similar_song_indices][[
        'track_name', 'artists', 'album_name', 'track_id', 'popularity']]

    return content_based_recommendations


In [10]:
def hybrid_recommendations(input_song_name, input_artist_name, num_recommendations=5, alpha=0.5):

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(
        input_song_name, input_artist_name, num_recommendations)

    if content_based_rec is None:
        return

    # Get the indices of the songs in the dataset that match the input song name.
    song = (df['track_name'].str.contains(
        input_song_name, case=False, na=False, regex=False))
    art = (df['artists'].str.contains(
        input_artist_name, case=False, na=False, regex=False))

    matching_song_indices = df[song & art].index[0]
    # # Get the popularity score of the input song
    popularity_score = df.loc[matching_song_indices, 'popularity']
    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        df.loc[matching_song_indices, 'release'])
    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = pd.concat([hybrid_recommendations,  pd.DataFrame.from_records([{
        'track_name': input_song_name,
        'artists': df.loc[df['track_name'] == input_song_name, 'artists'].values[0],
        'album_name': df.loc[df['track_name'] == input_song_name, 'album_name'].values[0],
        'track_id': df.loc[df['track_name'] == input_song_name, 'track_id'].values[0],
        'popularity': weighted_popularity_score
    }])])

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(
        by='popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['track_name'] != input_song_name]

    return hybrid_recommendations

In [16]:
#Test the function
def printout(song_name, artist_name, num_recommendations=6):
    recommendations = hybrid_recommendations(
        song_name, artist_name, num_recommendations)
    print(f"Hybrid recommended songs for '{song_name}':")
    print(recommendations)
    return recommendations

printout('Into The Night', 'YOASOBI')

Hybrid recommended songs for 'Into The Night':
                                              track_name          artists  \
14398                           Dancing in the Moonlight        Toploader   
20769                                    I'm Every Woman       Chaka Khan   
83966                                              ハルジオン          YOASOBI   
10726                           Dancing in the Moonlight        Toploader   
28518  Do It All Night (Michael Mind Remix) [feat. Ni...  Darius & Finlay   

                                            album_name  \
14398  Dancing In The Moonlight: The Best Of Toploader   
20769                                            Chaka   
83966                                            ハルジオン   
10726                                  Onka's Big Moka   
28518                           Euro Club Hits Vol. 11   

                     track_id  popularity  
14398  1FHNctV68GUNLgXclG2DtR        67.0  
20769  1oFiPGBafH9Woo9AMwgBSl        66.0  
83966  7H

,track_name,artists,album_name,track_id,popularity
14398,Dancing in the Moonlight,Toploader,Dancing In The Moonlight: The Best Of Toploader,1FHNctV68GUNLgXclG2DtR,67.0
20769,I'm Every Woman,Chaka Khan,Chaka,1oFiPGBafH9Woo9AMwgBSl,66.0
83966,ハルジオン,YOASOBI,ハルジオン,7HneEBTvTra2CRYsxgMOAi,62.0
10726,Dancing in the Moonlight,Toploader,Onka's Big Moka,0w07a1vsKahQMM0RnPXHVT,53.0
28518,Do It All Night (Michael Mind Remix) [feat. Ni...,Darius & Finlay,Euro Club Hits Vol. 11,2TvkDAlQzIZ6wlgtZhUwrD,28.0
